In [ ]:
import torchvision
import os
os.environ['TORCH_HOME']=r'E:\VScodes\ipython\深度学习基础\神经网络\现有模型的使用和修改'

vgg16_false = torchvision.models.vgg16(pretrained=False)

vgg16_true = torchvision.models.vgg16(pretrained=True)

print(vgg16_true)

In [ ]:
from  torch.nn import Linear
train_data = torchvision.datasets.CIFAR10("dataset",train=True,transform=torchvision.transforms.ToTensor(),download=True)

# 添加网络层
# 使得最后的输出是10类
vgg16_true.classifier.add_module("add_linear",Linear(1000,10))
vgg16_true.classifier[6] = Linear(4096,10)

In [ ]:
# 模型的保存和加载
# 使用神经网络基本骨架那一篇最后设置的神经网络框架
import torch
import torchvision
from torch import  nn
from torch.nn import ReLU,Sigmoid,Linear,Sequential,Conv2d,MaxPool2d,Flatten
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
# 取测试集数据，并将其转换为tensor数据类型
dataset = torchvision.datasets.CIFAR10("dataset",train=False,transform=torchvision.transforms.ToTensor(),download=True)

dataloader = DataLoader(dataset,batch_size=1)

class Seq(nn.Module):
    def __init__(self):
        super(Seq,self).__init__()
        self.model = nn.Sequential(
            Conv2d(3,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,32,5,padding=2),
            MaxPool2d(2),
            Conv2d(32,64,5,padding=2),
            MaxPool2d(2),
            Flatten(),
            Linear(1024,64),
            Linear(64,10)
        )
    
    def forward(self,x):
        x = self.model(x)
        return x

seq = Seq()
# 第一种，保存模型和参数
torch.save(seq,"Seq.pth")

# 加载模型
model = torch.load("Seq.pth")

# 第二种 只保存参数，但是加载的时候要求存在类的定义
torch.save(seq.state_dict(),"Seq_dic.pth")
model1 = Seq()
model1.load_state_dict(torch.load("Seq_dic.pth"))
print(model1)


In [ ]:
import torch
import time
import torchvision
import torch.nn
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch.nn import Flatten
from torch.utils.tensorboard import SummaryWriter

train_data = torchvision.datasets.CIFAR10("dataset",train = True,transform=torchvision.transforms.ToTensor(),download=True)
test_data = torchvision.datasets.CIFAR10("dataset",train = False,transform=torchvision.transforms.ToTensor(),download=True)
# 批量加载
train_dataloader = DataLoader(train_data,batch_size=64)
test_dataloader = DataLoader(test_data,batch_size=64)
test_size = len(test_data)

# 使用已经使用imagenet训练好的vgg16模型
vgg16 = torchvision.models.vgg16(pretrained=True)
vgg16.classifier.add_module("add_linear",Linear(1000,10))
vgg16 = vgg16.cuda()
# 定义损失函数和优化器
loss_func = torch.nn.CrossEntropyLoss()
loss_func = loss_func.cuda()

learn_rate = 1e-2
optimizer = torch.optim.SGD(vgg16.parameters(),lr=learn_rate)


# 训练次数 和 测试次数
train_total = 0
test_total = 0
# 训练的轮数  这里设置为10
# 后期可以尝试 30 50 查看效果。
epoch = 10

# 使用tensorboard查看效果
writer = SummaryWriter()

for i in range(epoch):
    
    start = time.time()
    print(f"------第{i}轮开始了------")
    for data in train_dataloader:
        imgs,targets = data
        imgs = imgs.cuda()
        targets = targets.cuda()
        ouput = vgg16(imgs)
        loss = loss_func(ouput,targets)

        # 优化器优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_total += 1
        if train_total % 500 ==0:
            end = time.time()
            print(f"训练次数{train_total},时间：{end-start}")
            writer.add_scalar("train_loss",loss.item(),train_total)
    # 测试步骤开始
    vgg16.eval()

    # 对于测试 ， 定义两个指标，一个是总损失情况，一个是准确率
    total_loss = 0.0
    total_accrancy = 0.0

    with torch.no_grad():
        for data in test_dataloader:
            imgs,targets = data
            imgs = imgs.cuda()
            targets = targets.cuda()
            ouput = vgg16(imgs)

            loss = loss_func(ouput,targets)
            total_loss += loss.item()

            accurancy = (ouput.argmax(1)==targets).sum()
            total_accrancy += accurancy
        accurancy1 = total_accrancy/test_size
        print(f"测试集上的总loss {total_loss}")
        print(f"测试集上的正确率 {accurancy1}")
        writer.add_scalar("test_loss",total_loss,i)
        writer.add_scalar("accrancy",accurancy1,i)
    
torch.save(vgg16,"vgg16_10.pth")


##训练结果，loss一步一步小，正确率一步一步大，说明正常。

##我这里最后训练的准确率在87%左右。